# Crawling Twitter Data based on **Search Keyword** Using `tweet-harvest`
Last updated by Helmi Satria (helmisatria.com) on December 12, 2023

Note: **For educational purposes only**


In [1]:
# # Install Node.js (because tweet-harvest built using Node.js)
# !sudo apt-get update
# !sudo apt-get install -y ca-certificates curl gnupg
# !sudo mkdir -p /etc/apt/keyrings
# !curl -fsSL https://deb.nodesource.com/gpgkey/nodesource-repo.gpg.key | sudo gpg --dearmor -o /etc/apt/keyrings/nodesource.gpg

# !NODE_MAJOR=20 && echo "deb [signed-by=/etc/apt/keyrings/nodesource.gpg] https://deb.nodesource.com/node_$NODE_MAJOR.x nodistro main" | sudo tee /etc/apt/sources.list.d/nodesource.list

# !sudo apt-get update
# !sudo apt-get install nodejs -y

# !node -v

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,611 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,606 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/univers

In [5]:
# # Crawl Data

# filename = 'pemilu.csv'
# search_keyword = '#DebatCapres until:2024-01-08 since:2024-01-06'
# limit = 3000

# !npx --yes tweet-harvest@2.2.8 -o "{filename}" -s "{search_keyword}" -l {limit} --token ""


Welcome to the Twitter Crawler 🕷️

This script uses Chromium Browser to crawl data from Twitter with *your* Twitter auth token.
Please enter your Twitter auth token when prompted.

Note: Keep your access token secret! Don't share it with anyone else.
Note: This script only runs on your local device.

? What's your Twitter auth token? › 78? What's your Twitter auth token? › *78? What's your Twitter auth token? › **78? What's your Twitter auth token? › ***78? What's your Twitter auth token? › ****78? What's your Twitter auth token? › *****78? What's your Twitter auth token? › ******78? What's your Twitter auth token? › *******78? What's your Twitter auth token? › ********78? What's your Twitter auth token? › *********78? What's your Twitter auth token? › **********78? What's your Twitter auth token? › ***********78? What's your Twitter auth token? › ************78? What's your Twitter auth token? › *************78? What's your Twitter auth token? › **********

In [1]:
import pandas as pd
import nltk, os, re, string
import zipfile

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, LSTM, Bidirectional, SpatialDropout1D, Dropout, Flatten, Dense, Embedding, BatchNormalization
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nlp_id.lemmatizer import Lemmatizer

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to C:\Users\asus.LAPTOP-
[nltk_data]     P9TBK6TS.000\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to C:\Users\asus.LAPTOP-
[nltk_data]     P9TBK6TS.000\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to C:\Users\asus.LAPTOP-
[nltk_data]     P9TBK6TS.000\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to C:\Users
[nltk_data]     \asus.LAPTOP-P9TBK6TS.000\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [2]:
df = pd.read_csv('pemilu (3).csv', delimiter=";")

display(df)

,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url
0,Sun Jan 07 23:55:32 +0000 2024,1744145777877037180,jiahahaha prabowo dicebokin bot #DebatCapres,0,0,0,1,in,1431142289884135428,1744145777877037180,hochidubu,https://twitter.com/hochidubu/status/174414577...
1,Sun Jan 07 23:54:50 +0000 2024,1744145600130711777,Biar masyarakat yg menilai ya pak. #02KubuArog...,0,0,0,2,in,1723961822829809664,1744145600130711777,archie99634,https://twitter.com/archie99634/status/1744145...
2,Sun Jan 07 23:54:32 +0000 2024,1744145523194638522,@erlanishere @Gerindra Dan diungkap di #DebatC...,0,0,0,0,in,604333557,1744004360651292983,rektrektrekt123,https://twitter.com/rektrektrekt123/status/174...
3,Sun Jan 07 23:53:34 +0000 2024,1744145279849542072,Jubir TKD Jatim: Prabowo akan Memperkuat Siste...,0,0,0,0,in,21287066,1744145279849542072,beritajatimcom,https://twitter.com/beritajatimcom/status/1744...
4,Sun Jan 07 23:52:51 +0000 2024,1744145100048077090,@RadioElshinta ANIES ROCKS DEBAT CAPRES !! #D...,0,0,1,3,en,129750289,1743960193225150558,ZAEffendy,https://twitter.com/ZAEffendy/status/174414510...
...,...,...,...,...,...,...,...,...,...,...,...,...
560,Sun Jan 07 21:57:32 +0000 2024,1744116079222501735,Keunggulan teknologi bukanlah pilihan tapi keh...,0,0,0,0,in,1725075851006664705,1744116079222501735,zarahanara,https://twitter.com/zarahanara/status/17441160...
561,Sun Jan 07 21:57:31 +0000 2024,1744116078589169724,Ganjar-Mahfud yakin investasi pada alutsista t...,0,0,0,0,in,1726072719366934528,1744116078589169724,rizkiirwas,https://twitter.com/rizkiirwas/status/17441160...
562,Sun Jan 07 21:57:31 +0000 2024,1744116077779673144,Pembelian alutsista sesuai kebutuhan masa kini...,0,0,0,0,in,1725062133904490496,1744116077779673144,raraaisyahh,https://twitter.com/raraaisyahh/status/1744116...
563,Sun Jan 07 21:57:27 +0000 2024,1744116060465586624,Alutsista yg terkini performa yg luar biasa @i...,0,0,0,0,in,1717020263181946880,1744116060465586624,rianamstk,https://twitter.com/rianamstk/status/174411606...


In [3]:
data = df[['created_at', 'username', 'full_text']]

data.head(10)

,created_at,username,full_text
0,Sun Jan 07 23:55:32 +0000 2024,hochidubu,jiahahaha prabowo dicebokin bot #DebatCapres
1,Sun Jan 07 23:54:50 +0000 2024,archie99634,Biar masyarakat yg menilai ya pak. #02KubuArog...
2,Sun Jan 07 23:54:32 +0000 2024,rektrektrekt123,@erlanishere @Gerindra Dan diungkap di #DebatC...
3,Sun Jan 07 23:53:34 +0000 2024,beritajatimcom,Jubir TKD Jatim: Prabowo akan Memperkuat Siste...
4,Sun Jan 07 23:52:51 +0000 2024,ZAEffendy,@RadioElshinta ANIES ROCKS DEBAT CAPRES !! #D...
5,Sun Jan 07 23:52:16 +0000 2024,ShireenEsa90311,"Israel, claims humanity and is saddened by the..."
6,Sun Jan 07 23:52:05 +0000 2024,AminudinRashya,Tetap semangat Jepang Barat Korea Fansite Indo...
7,Sun Jan 07 23:51:07 +0000 2024,ceasarru,Yang mendesak El Gemoy itu baru satu orang saj...
8,Sun Jan 07 23:50:39 +0000 2024,radenimanuel1,Itu pendukung pak anies apa ga ada yang hilang...
9,Sun Jan 07 23:50:06 +0000 2024,maul_af,Ada yang pernah ngerasain seperti ini? 😭 - A ...


In [23]:
from langdetect import detect

def detect_language(text):
    try:
        lang = detect(text)
        return lang
    except:
        return 'unknown'

# Menambahkan kolom baru untuk menyimpan hasil deteksi bahasa
data['Language'] = data['full_text'].apply(detect_language)

# Memfilter tweet yang bahasa Indonesiannya terdeteksi
data = data[data['Language'] == 'id']

In [13]:
data['full_text'].head(10)

0         jiahahaha prabowo dicebokin bot  #DebatCapres
1     Biar masyarakat yg menilai ya pak. #02KubuArog...
2     @erlanishere @Gerindra Dan diungkap di #DebatC...
3     Jubir TKD Jatim: Prabowo akan Memperkuat Siste...
4     @RadioElshinta ANIES ROCKS DEBAT CAPRES !!  #D...
6     Tetap semangat Jepang Barat Korea Fansite Indo...
7     Yang mendesak El Gemoy itu baru satu orang saj...
8     Itu pendukung pak anies apa ga ada yang hilang...
9     Ada yang pernah ngerasain seperti ini? 😭  - A ...
11    Promosi RI di Dunia, Ganjar Mau Pakai Viralism...
Name: full_text, dtype: object

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 409 entries, 0 to 564
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  409 non-null    object
 1   username    409 non-null    object
 2   full_text   409 non-null    object
 3   Language    409 non-null    object
dtypes: object(4)
memory usage: 16.0+ KB


In [15]:
data.shape

(409, 4)

In [17]:
import re

def remove_tweet_special(text):
    # Menghapus tab, baris baru, dan back slice
    text = text.replace('\\t', ' ').replace('\\n', ' ').replace('\\u', ' ').replace('\\', '')

    # Menghapus karakter non-ASCII (emotikon, huruf Cina, dll.)
    text = text.encode('ascii', 'replace').decode('ascii')

    # Menghapus mention, link, dan hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", " ", text).split())

    # Menghapus URL yang tidak lengkap
    text = text.replace("http://", " ").replace("https://", " ")

    return text

def clean_text(data):
    # Menghapus karakter yang tidak diinginkan (_) dan karakter selain huruf
    data['full_text'] = data['full_text'].str.replace(r"[^a-zA-Z ]+", " ").str.strip()

    # Mengganti multiple whitespace dengan single whitespace
    data['full_text'] = data['full_text'].replace(" +", " ", regex=True)

    return data

# Menggunakan fungsi remove_tweet_special untuk membersihkan teks
data['full_text'] = data['full_text'].apply(remove_tweet_special)

# Menggunakan fungsi clean_text untuk membersihkan karakter yang tidak diinginkan
data = clean_text(data)


In [18]:
def clean_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

lemmatizer = Lemmatizer()

def lemmatize_text(text):
    return lemmatizer.lemmatize(text)

def remove_numbers(text):
    return re.sub('[0-9]+', '', text)

def remove_stopwords(text):
    stop_words = set(stopwords.words('indonesian'))
    return ' '.join([word for word in nltk.word_tokenize(text) if word.lower() not in stop_words])

data['full_text'] = data['full_text'].apply(lambda x: x.lower())

data['full_text'] = data['full_text'].apply(clean_punctuation)

data['full_text'] = data['full_text'].apply(lemmatize_text)

data['full_text'] = data['full_text'].apply(remove_numbers)

data['full_text'] = data['full_text'].apply(remove_stopwords)

In [24]:
data.head(10)

,created_at,username,full_text,Language
1,Sun Jan 07 23:54:50 +0000 2024,archie99634,biar masyarakat yg nilai ya,id
3,Sun Jan 07 23:53:34 +0000 2024,beritajatimcom,jubir tkd jatim prabowo kuat sistem militer in...,id
6,Sun Jan 07 23:52:05 +0000 2024,AminudinRashya,semangat jepang barat korea fansite indonesia ...,id
7,Sun Jan 07 23:51:07 +0000 2024,ceasarru,desak el gemoy orang amp beliau males salam ma...,id
8,Sun Jan 07 23:50:39 +0000 2024,radenimanuel1,dukung anies ga hilang respect gara gara nyeng...,id
12,Sun Jan 07 23:49:42 +0000 2024,Saodah1234,orang prajurit tni sertu afriadi gugur tembak ...,id
13,Sun Jan 07 23:49:38 +0000 2024,fajarwahyu999,bikin event pakai undangin yuk acara cocok ban...,id
14,Sun Jan 07 23:49:25 +0000 2024,haloswant,debat kemarin seru sayang serang lontar menhan...,id
17,Sun Jan 07 23:48:03 +0000 2024,Anaxangels86,lihat selamat koya sdh tau karakter orang yg yg,id
18,Sun Jan 07 23:47:39 +0000 2024,_samimawon,selamat pagi warga xlandgmn selesai nonton deb...,id


In [25]:
!pip install Sastrawi


     ---------------------------------------- 0.0/209.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/209.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/209.7 kB ? eta -:--:--
     ---------------------------------------- 0.0/209.7 kB ? eta -:--:--
     ----- ---------------------------------- 30.7/209.7 kB ? eta -:--:--
     ----- ---------------------------------- 30.7/209.7 kB ? eta -:--:--
     ----- ---------------------------------- 30.7/209.7 kB ? eta -:--:--
     ----- ---------------------------------- 30.7/209.7 kB ? eta -:--:--
     ------------------- ---------------- 112.6/209.7 kB 547.6 kB/s eta 0:00:01
     ------------------------------------ 209.7/209.7 kB 852.3 kB/s eta 0:00:00


In [27]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Inisialisasi Stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi untuk melakukan stemming pada setiap teks dalam kolom 'full_text'
def stem_text(text):
    return stemmer.stem(text)

# Melakukan stemming pada kolom 'full_text'
data['full_text_stemmed'] = data['full_text'].apply(stem_text)

# Menampilkan hasil stemming
data.head(10)


,created_at,username,full_text,Language,full_text_stemmed
1,Sun Jan 07 23:54:50 +0000 2024,archie99634,biar masyarakat yg nilai ya,id,biar masyarakat yg nilai ya
3,Sun Jan 07 23:53:34 +0000 2024,beritajatimcom,jubir tkd jatim prabowo kuat sistem militer in...,id,jubir tkd jatim prabowo kuat sistem militer in...
6,Sun Jan 07 23:52:05 +0000 2024,AminudinRashya,semangat jepang barat korea fansite indonesia ...,id,semangat jepang barat korea fansite indonesia ...
7,Sun Jan 07 23:51:07 +0000 2024,ceasarru,desak el gemoy orang amp beliau males salam ma...,id,desak el gemoy orang amp beliau males salam ma...
8,Sun Jan 07 23:50:39 +0000 2024,radenimanuel1,dukung anies ga hilang respect gara gara nyeng...,id,dukung anies ga hilang respect gara gara nyeng...
12,Sun Jan 07 23:49:42 +0000 2024,Saodah1234,orang prajurit tni sertu afriadi gugur tembak ...,id,orang prajurit tni sertu afriadi gugur tembak ...
13,Sun Jan 07 23:49:38 +0000 2024,fajarwahyu999,bikin event pakai undangin yuk acara cocok ban...,id,bikin event pakai undangin yuk acara cocok ban...
14,Sun Jan 07 23:49:25 +0000 2024,haloswant,debat kemarin seru sayang serang lontar menhan...,id,debat kemarin seru sayang serang lontar menhan...
17,Sun Jan 07 23:48:03 +0000 2024,Anaxangels86,lihat selamat koya sdh tau karakter orang yg yg,id,lihat selamat koya sdh tau karakter orang yg yg
18,Sun Jan 07 23:47:39 +0000 2024,_samimawon,selamat pagi warga xlandgmn selesai nonton deb...,id,selamat pagi warga xlandgmn selesai nonton deb...


In [28]:
tokenizer = Tokenizer(num_words=5000, oov_token='x', filters='!"#$%&()*+,-./:;<=>@[\]^_`{|}~ ')

# Menggunakan fit_on_texts untuk mengonversi teks menjadi token
tokenizer.fit_on_texts(data['full_text'])

# Konversi teks menjadi sequence
data_sequences = tokenizer.texts_to_sequences(data['full_text'])

# Menampilkan hasil sequence
print(data_sequences)

[[85, 86, 3, 134, 20], [216, 325, 521, 10, 31, 217, 163, 5, 326, 35, 216, 218, 522, 523, 325, 524, 525, 526, 527, 10, 528, 529], [135, 327, 530, 328, 531, 5, 532, 533, 219, 534, 535, 536, 537, 538, 539, 540, 541, 542], [543, 111, 99, 44, 87, 164, 220, 165, 220, 329, 166, 220, 544, 112, 111, 99, 221, 74, 222, 87, 330, 58, 222, 223, 24, 20, 167, 87, 24, 330], [36, 21, 63, 224, 545, 331, 331, 546, 225, 51, 64, 547], [44, 332, 226, 548, 549, 550, 551, 552, 553, 8, 554, 333, 555, 556, 557, 334, 558, 333, 559, 560, 561, 562, 13, 65, 27, 113, 25], [227, 563, 228, 564, 565, 168, 566, 229, 567, 568, 335], [6, 336, 169, 337, 33, 569, 66, 65, 136, 570, 338, 170, 571, 572, 339, 230, 231, 232, 233, 113, 13], [38, 234, 573, 235, 100, 236, 44, 3, 3], [234, 340, 237, 574, 341, 67, 6, 575, 576, 577, 578, 579, 88, 59, 21, 342, 6, 17], [171, 114, 17, 343, 238, 239, 171, 114, 6, 17, 36, 238, 239, 219, 115], [89, 580, 581, 137, 582, 583, 584, 89, 585], [586, 3, 67, 240, 13, 587, 588, 589, 20, 240, 60, 65, 

In [31]:
file_negative = 'InSet/negative.tsv'
file_positive = 'Inset/positive.tsv'

data_negative = pd.read_csv(file_negative, sep='\t')
data_positive = pd.read_csv(file_positive, sep='\t')

# Tambahkan kolom label sentimen
data_negative['sentiment'] = 'negative'
data_positive['sentiment'] = 'positive'

# Gabungkan dataset negative dan positive menjadi satu dataset
data_combined = pd.concat([data_negative, data_positive], ignore_index=True)

# Tampilkan beberapa contoh data
data_combined.head(10)


,word,weight,sentiment
0,putus tali gantung,-2,negative
1,gelebah,-2,negative
2,gobar hati,-2,negative
3,tersentuh (perasaan),-1,negative
4,isak,-5,negative
5,larat hati,-3,negative
6,nelangsa,-3,negative
7,remuk redam,-5,negative
8,tidak segan,-2,negative
9,gemar,-1,negative


In [ ]:
file_negative = 'InSet/negative.tsv'
file_positive = 'Inset/positive.tsv'

data_negative = pd.read_csv(file_negative, sep='\t')
data_positive = pd.read_csv(file_positive, sep='\t')

# Tambahkan kolom label sentimen
data_negative['sentiment'] = 'negative'
data_positive['sentiment'] = 'positive'

# Gabungkan dataset negative dan positive menjadi satu dataset
data_combined = pd.concat([data_negative, data_positive], ignore_index=True)

# Tampilkan beberapa contoh data
data_combined.head(10)


,word,weight,sentiment
0,putus tali gantung,-2,negative
1,gelebah,-2,negative
2,gobar hati,-2,negative
3,tersentuh (perasaan),-1,negative
4,isak,-5,negative
5,larat hati,-3,negative
6,nelangsa,-3,negative
7,remuk redam,-5,negative
8,tidak segan,-2,negative
9,gemar,-1,negative


In [ ]:
file_negative = 'InSet/negative.tsv'
file_positive = 'Inset/positive.tsv'

data_negative = pd.read_csv(file_negative, sep='\t')
data_positive = pd.read_csv(file_positive, sep='\t')

# Tambahkan kolom label sentimen
data_negative['sentiment'] = 'negative'
data_positive['sentiment'] = 'positive'

# Gabungkan dataset negative dan positive menjadi satu dataset
data_combined = pd.concat([data_negative, data_positive], ignore_index=True)

# Tampilkan beberapa contoh data
data_combined.head(10)


,word,weight,sentiment
0,putus tali gantung,-2,negative
1,gelebah,-2,negative
2,gobar hati,-2,negative
3,tersentuh (perasaan),-1,negative
4,isak,-5,negative
5,larat hati,-3,negative
6,nelangsa,-3,negative
7,remuk redam,-5,negative
8,tidak segan,-2,negative
9,gemar,-1,negative


In [ ]:
file_negative = 'InSet/negative.tsv'
file_positive = 'Inset/positive.tsv'

data_negative = pd.read_csv(file_negative, sep='\t')
data_positive = pd.read_csv(file_positive, sep='\t')

# Tambahkan kolom label sentimen
data_negative['sentiment'] = 'negative'
data_positive['sentiment'] = 'positive'

# Gabungkan dataset negative dan positive menjadi satu dataset
data_combined = pd.concat([data_negative, data_positive], ignore_index=True)

# Tampilkan beberapa contoh data
data_combined.head(10)


,word,weight,sentiment
0,putus tali gantung,-2,negative
1,gelebah,-2,negative
2,gobar hati,-2,negative
3,tersentuh (perasaan),-1,negative
4,isak,-5,negative
5,larat hati,-3,negative
6,nelangsa,-3,negative
7,remuk redam,-5,negative
8,tidak segan,-2,negative
9,gemar,-1,negative
